In [1]:
import os
import sys

In [2]:
%pwd
os.chdir('..')

In [3]:
%pwd

'd:\\Self-Pace\\ML-Projects\\Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    SOURCE_URL: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
from TextSummarizer.constants import *
from TextSummarizer.utils.utils import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    
    def __init__(self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH,
                ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            SOURCE_URL=config.SOURCE_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [7]:
import os
import urllib.request as request
import zipfile

from TextSummarizer.logging import logging
from TextSummarizer.utils.utils import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
            filename, headers = request.urlretrieve(
                url = self.config.SOURCE_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    logging.error(f"Error in data ingestion: {e}")
    sys.exit(1)

[2024-05-02 13:20:33,264] 18 - INFO  - Initiating read_yaml function
[2024-05-02 13:20:33,274] 22 - INFO  - yaml file:config\config.yaml loaded Successfully
[2024-05-02 13:20:33,278] 18 - INFO  - Initiating read_yaml function
[2024-05-02 13:20:33,284] 22 - INFO  - yaml file:params\params.yaml loaded Successfully
[2024-05-02 13:20:33,287] 39 - INFO  - Initiating create_directories function
[2024-05-02 13:20:33,292] 44 - INFO  - Created directory artifacts
[2024-05-02 13:20:34,815] 12 - INFO  - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 63BE:CA401:13F644:1B0172:66332F23